In [ ]:
import os
from langchain_core.prompts import ChatPromptTemplate
import re
import json
import numpy as np

In [3]:
from google import genai

client = genai.Client(api_key=api_key)

```
Input               Agent                     Output
Query           -> Filter ->      Từ khoá (Subject, chapter, ...)
Query + Chunks  -> LLMs trả lời ->          Kết quả cuối cùng
```

## ***Prompt dùng để trả về từ khoá để thực hiện metadata filtering***

In [4]:
def get_keywords(query):
    system_prompt =""" 
        Bạn là một con Agent sẽ thực hiện filter các từ khoá quan trọng thông qua các từ khoá quy định sau đây.
        
        Các từ khoá quy định: 
            type.
        
        Chi tiết các từ khoá:
            - Type: Phân loại nội dung, sẽ có 3 kiểu "TARGET", "EXERCISES", "THEORY". Trong đó:
                + "TARGET": Là những nội dung chính của chương, phần, tiểu mục.
                + "EXERCISES": Là những bài tập, câu hỏi liên quan đến chương
                + "THEORY": Là những lý thuyết, kiến thức nền tảng liên quan đến chương, phần, tiểu mục.

        Đây là câu query của người dùng: 
        {query}
        
        ** Lưu ý:
        - Chỉ trả về các từ khoá quan trọng theo định dạng json. Kết quả trả về sẽ là một đối tượng JSON với các trường tương ứng với các từ khoá đã nêu ở trên. Không bao gồm những nội dung không cần thiết.
        - Ví dụ kết quả trả về:
        {{
            "type": {{"$eq": "THEORY"}}
        }}
        - Với các từ khoá trên, với mỗi từ khoá tương ứng, nếu không tìm được thông tin thì không cần trả về từ khoá.
    """

    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
    ])

    messages = prompt_template.format_messages(
        query=query.strip()
    )
    # "Chương nào của môn Lịch sử Đảng Cộng sản Việt Nam có nội dung về sự hình thành của Đảng Cộng sản Việt Nam?"
    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents=messages[0].content.strip()
    )
    
    clean_text = re.sub(r"```json|```", "", response.text.strip()).strip()
    parsed = json.loads(clean_text)
    return parsed

## ***Model embedding***

In [5]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("AITeamVN/Vietnamese_Embedding")

C:\Users\Duongw\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## ***Gọi database***

In [21]:
query = "Cho tôi một vài câu hỏi luyện tập môn lịch sử đảng của chương 2"
keywords = get_keywords(query)
keywords

{'subject': {'$eq': 'Lịch sử Đảng Cộng sản Việt Nam'},
 'type': {'$eq': 'EXERCISES'}}

In [24]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_key)

# To get the unique host for an index, 
# see https://docs.pinecone.io/guides/manage-data/target-an-index
dense_index = pc.Index(host="https://rag-ktcn-0abpzwm.svc.aped-4627-b74a.pinecone.io")
sparse_index = pc.Index(host="https://ktcn-rag-sparse-0abpzwm.svc.aped-4627-b74a.pinecone.io")

### ***Load raw chunks***

In [7]:
def load_chunks_from_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        chunks = json.load(f)
    return chunks

In [8]:
raw_path = "..\data\LichSuDang\Lich_Su_Dang_raw.json"
raw_chunk = load_chunks_from_json(raw_path)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Duongw\AppData\Local\Temp\ipykernel_21428\1837902065.py:1: SyntaxWarning: invalid escape sequence '\d'
  raw_path = "..\data\LichSuDang\Lich_Su_Dang_raw.json"


## ***Sparse vector converter***

In [11]:
def bm25_tokenize(text):
    return text.lower().split()

def text_to_sparse_vector_bm25(text, bm25, vocabulary):
    tokens = bm25_tokenize(text)
    vector = np.zeros(len(vocabulary))
    for i, word in enumerate(vocabulary):
        idf = bm25.idf.get(word, 0)
        tf = tokens.count(word)
        vector[i] = idf * tf
    indices = vector.nonzero()[0].tolist()
    values = vector[indices].tolist()
    return {"indices": indices, "values": values}

In [17]:
from rank_bm25 import BM25Okapi
import numpy as np

# Tạo corpus
corpus_texts = [chunk["content"] for chunk in raw_chunk]
tokenized_corpus = [bm25_tokenize(text) for text in corpus_texts]

bm25 = BM25Okapi(tokenized_corpus)
vocabulary = list(bm25.idf.keys())

In [67]:
query = """Trình bày những biện pháp nhân nhượng của quân ta đối với quân Tưởng?"""
sparse_vector = text_to_sparse_vector_bm25(query, bm25, vocabulary)

In [68]:
sparse_results = sparse_index.query(
    namespace="lich-su-dang",
    sparse_vector=sparse_vector,
    top_k=10,
    include_metadata=True,
)
sparse_results

{'matches': [{'id': 'LSD_chuong2_I_1_8',
              'metadata': {'chapter': 'Chương 2',
                           'chapter_title': 'ĐẢNG LÃNH ĐẠO HAI CUỘC KHÁNG '
                                            'CHIẾN, HOÀN THÀNH GIẢI PHÓNG DÂN '
                                            'TỘC, THỐNG NHẤT ĐẤT NƯỚC (1945 - '
                                            '1975)',
                           'content': '. Để tránh mũi nhọn tấn công của các kẻ '
                                      'thù, Đảng chủ trương rút vào hoạt động '
                                      'bí mật bằng việc ra “Thông cáo Đảng '
                                      'Cộng sản Đông Dương tự ý tự giải tán, '
                                      'ngày 11-11-1945”, chỉ để lại một bộ '
                                      'phận hoạt động công khai với danh nghĩa '
                                      '“Hội nghiên cứu chủ nghĩa Mác ở Đông '
                                      'Dương”; Chính phủ Việt Nam đ

## ***Dense vector converter***

In [69]:
embed_query = embedding_model.encode(query, convert_to_tensor=False).tolist()
# keywords = get_keywords(query)

In [70]:
dense_results = dense_index.query(
    namespace="lich-su-dang", 
    vector=embed_query,
    top_k=10,
    include_metadata=True,
    # fields=["category", "chunk_text"]
)
print(dense_results)

{'matches': [{'id': 'LSD_chuong2_I_1_8',
              'metadata': {'chapter': 'Chương 2',
                           'chapter_title': 'ĐẢNG LÃNH ĐẠO HAI CUỘC KHÁNG '
                                            'CHIẾN, HOÀN THÀNH GIẢI PHÓNG DÂN '
                                            'TỘC, THỐNG NHẤT ĐẤT NƯỚC (1945 - '
                                            '1975)',
                           'content': '. Để tránh mũi nhọn tấn công của các kẻ '
                                      'thù, Đảng chủ trương rút vào hoạt động '
                                      'bí mật bằng việc ra “Thông cáo Đảng '
                                      'Cộng sản Đông Dương tự ý tự giải tán, '
                                      'ngày 11-11-1945”, chỉ để lại một bộ '
                                      'phận hoạt động công khai với danh nghĩa '
                                      '“Hội nghiên cứu chủ nghĩa Mác ở Đông '
                                      'Dương”; Chính phủ Việt Nam đ

In [71]:
dense_results['matches']

[{'id': 'LSD_chuong2_I_1_8',
  'metadata': {'chapter': 'Chương 2',
               'chapter_title': 'ĐẢNG LÃNH ĐẠO HAI CUỘC KHÁNG CHIẾN, HOÀN THÀNH '
                                'GIẢI PHÓNG DÂN TỘC, THỐNG NHẤT ĐẤT NƯỚC (1945 '
                                '- 1975)',
               'content': '. Để tránh mũi nhọn tấn công của các kẻ thù, Đảng '
                          'chủ trương rút vào hoạt động bí mật bằng việc ra '
                          '“Thông cáo Đảng Cộng sản Đông Dương tự ý tự giải '
                          'tán, ngày 11-11-1945”, chỉ để lại một bộ phận hoạt '
                          'động công khai với danh nghĩa “Hội nghiên cứu chủ '
                          'nghĩa Mác ở Đông Dương”; Chính phủ Việt Nam đồng ý '
                          'việc đảm bảo cung cấp lương thực, thực phẩm cần '
                          'thiết cho 20 vạn quân đội Tưởng khi ở Việt Nam và '
                          'nhân nhượng cho quân Tưởng được sử dụng đồng tiền '
                  

In [ ]:
def merge_chunks(h1, h2):
    """Get the unique hits from two search results and return them as single array of {'_id', 'chunk_text'} dicts, printing each dict on a new line."""
    # Deduplicate by _id
    deduped_hits = {hit['id']: hit for hit in h1['matches'] + h2['matches']}.values()
    print
    # Sort by _score descending
    sorted_hits = sorted(deduped_hits, key=lambda x: x['score'], reverse=True)
    # Transform to format for reranking
    result = [{'id': hit['id'], 'content': hit['metadata']['content']} for hit in sorted_hits]
    return result

merged_results = merge_chunks(sparse_results, dense_results)

print('[\n   ' + ',\n   '.join(str(obj) for obj in merged_results) + '\n]')
print(len(merged_results))

[
   {'id': 'LSD_chuongnhapmon_II_2_0', 'content': 'Nhiệm vụ của khoa học lịch sử Đảng được đặt ra từ đối tượng nghiên cứu đồng thời cụ thể hóa chức năng của khoa học lịch sử Đảng. - Nhiệm vụ trình bày có hệ thống Cương lĩnh, đường lối của Đảng. Khoa học lịch sử Đảng có nhiệm vụ hàng đầu là khẳng định, chứng minh giá trị khoa học và hiện thực của những mục tiêu chiến lược và sách lược cách mạng mà Đảng đề ra trong Cương lĩnh, đường lối từ khi Đảng ra đời và suốt quá trình lãnh đạo cách mạng. Mục tiêu và con đường đó là sự kết hợp, thống nhất giữa thực tiễn lịch sử với nền tảng lý luận nhằm thúc đẩy tiến trình cách mạng, nhận thức và cải biến đất nước, xã hội theo con đường đúng đắn. Sự lựa chọn mục tiêu độc lập dân tộc gắn liền với chủ nghĩa xã hội phù hợp với quy luật tiến hóa của lịch sử, đã và đang được hiện thực hóa. - Nhiệm vụ tái hiện tiến trình lịch sử lãnh đạo, đấu tranh của Đảng. Từ hiện thực lịch sử và các nguồn tư liệu thành văn và không thành văn, khoa học lịch sử Đảng có n